In [37]:
import pandas as pd
import numpy as np
from math import isnan

DEBUG = 0

def readPaper(path, para_list:dict):
    print("Read Start:", path)
    f = open(path, mode='r', encoding='utf-8')
    # paper_dict = {}

    # 两个数据集
    filtered_paper = {"index":[]}
    for k in para_list:
        filtered_paper[para_list[k]] = []

    while True:
        index = f.readline()[7:].strip("\n")
        # 判断读到文件末尾
        if not index:
            print("Read End:", path)
            break
        tmp = {"index":[index]}
        while True:
            line = f.readline()
            # 每个对象空行为结尾
            if line == '\n':
                break
            line = line.strip("\n")
            for para in para_list:
                para_p = line.find(para)
                # 寻找参数
                if para_p != -1:
                    para_name = para_list[para]
                    if para_name not in tmp:
                        tmp[para_name] = []
                    tmp[para_name].append(line[len(para) + 1:])
                    break

        # 筛选数据
        if tmp['year'][0] <= '1993' and tmp['year'][0] >= '1984':
            continue
        for k in filtered_paper:
            try:
                filtered_paper[k].append("|".join(tmp[k]))
            except Exception as identifier:
                filtered_paper[k].append("")

        if DEBUG and len(filtered_paper['index']) == 20:
            # print(filtered_paper)
            break
    f.close()
    return filtered_paper
    # return paper_dict

def readAuthor2Paper(path)->pd.DataFrame:
    df = pd.read_csv(path, sep="\t")
    return df

def readAP10(path)->pd.DataFrame:
    df = pd.read_csv(path)
    return df

def readAuthor(path, para_list: dict):
    print("Read Start:", path)
    f = open(path, mode='r', encoding='utf-8')
    # paper_dict = {}

    # 两个数据集
    filtered_author = {"index": []}
    for k in para_list:
        filtered_author[para_list[k]] = []

    while True:
        index = f.readline()[7:].strip("\n")
        # 判断读到文件末尾
        if not index:
            print("Read End:", path)
            break
        tmp = {"index": [index]}
        while True:
            line = f.readline()
            # 每个对象空行为结尾
            if line == '\n':
                break
            line = line.strip("\n")
            for para in para_list:
                para_p = line.find(para)
                # 寻找参数
                if para_p != -1:
                    para_name = para_list[para]
                    if para_name not in tmp:
                        tmp[para_name] = []
                    tmp[para_name].append(line[len(para) + 1:])
                    break

        for k in filtered_author:
            try:
                filtered_author[k].append("|".join(tmp[k]))
            except Exception as identifier:
                filtered_author[k].append("")

        if DEBUG and len(filtered_author['index']) == 20:
            break
    f.close()
    return filtered_author

def readCoauthor(path):
    print("Read Start:", path)
    f = open(path, 'r', encoding='utf8')
    coauthor_dict = {'author1':[], 'author2':[], 'cop_time':[]}
    while True:
        line = f.readline()
        if not line:
            print("Read End:", path)
            break
        line = line[1:].strip('\n').split('\t')
        coauthor_dict['author1'].append(line[0])
        coauthor_dict['author2'].append(line[1])
        coauthor_dict['cop_time'].append(line[2])
    f.close()
    return coauthor_dict


In [38]:
# ap10和B-paper数据集
paper_para_list = {"#*": "title", "#@": "authors", "#o": "affiliations", "#t":"year", "#c":"publication_venue", "#%":"refs", "#!":"abstract"}
ap10 = readPaper("AMiner-Paper.txt", paper_para_list)
ap10 = pd.DataFrame(ap10)
ap10[['index']] = ap10[['index']].astype(np.int64)
ap10.to_csv("ap10.csv")

Read Start: AMiner-Paper.txt
Read End: AMiner-Paper.txt


In [40]:
AP10 = pd.read_csv("ap10.csv", dtype={'refs': str})
ref_re = AP10[['index', 'refs']]
# ref_dict = {}
# 被引字典
refed_dict = {}
for index, row in ref_re.iterrows():
    if isinstance(row['refs'], float):
        continue
    _ref_table = row['refs'].split('|')

    # ref_dict[row['index']]=_ref_table
    for _ref in _ref_table:
        if _ref not in refed_dict:
            refed_dict[_ref] = []
        refed_dict[_ref].append(row['index'])

C:\Users\t-jintli\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
print(len(refed_dict))
print(refed_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:

# # ap10和B-paper数据集
paper_para_list = {"#*": "title", "#@": "authors", "#o": "affiliations", "#t":"year", "#c":"publication_venue", "#%":"refs", "#!":"abstract"}
papers = readPaper("AMiner-Paper.txt", paper_para_list)
papers = pd.DataFrame(papers)
papers[['index']] = papers[['index']].astype(np.int64)
# # papers.to_csv("ap10.csv", index=False)

# 读取author数据集
author_para_list = {"#n": "name", "#a": "affiliations", "#pc": "paper_cnt",
                    "#cn": "cite_cnt", "#hi": "hi", "#pi": "pi", "#upi": "upi", "#t": "interest"}
authors = readAuthor("AMiner-Author.txt", author_para_list)
authors = pd.DataFrame(authors)
authors[['index']] = authors[['index']].astype(np.int64)

# # 学术新人数据集
# # Author Paper in 10 years (1984-1993)
# ap10 = readAP10("ap10.csv")

# # 提取1984年的论文
# P = ap10[ap10["year"] == 1984]
# Author to Paper
a2p = readAuthor2Paper("AMiner-Author2Paper.csv")
# # print(a2p)

# # 提取T：P中论文对应的作者集合
# # 用p中的paper_id进行连接，得出作者集合T
# a2p_join_P = pd.merge(a2p, P, left_on='paper_id', right_on='index')
# T = a2p_join_P[['author_id']]

# # 提取EPI: T中作者对应的论文集合
# # a2p_join_T = pd.merge(a2p, T, on='author_id')
# # EPi = a2p_join_T.drop_duplicates(['author_id', 'paper_id'])
# # EPi = EPi[['author_id', 'paper_id']]
# # EPi = pd.merge(EPi, papers, left_on='paper_id', right_on='index')[['author_id','paper_id','year']]
# # 保存到epi_with_years中
# # EPi.to_csv('epi_with_years.csv')
# # print(EPi)

# 读取已经处理完的epi_with_years
# EPi = pd.read_csv('epi_with_years.csv', sep=',')
# # print(EPi.dtypes)
# # 得到新人数据
# A = pd.DataFrame({'author_id': [], 'hi': [], 'EPi10_paper_id': []})
# # 在Paper中遍历T
# for idx, row in T.iterrows():
#     tmp_paper_ids = EPi.loc[EPi['author_id'] == row.author_id]
#     tmp_paper_years = tmp_paper_ids.year.unique()
#     is_A = ((tmp_paper_years >= 1984).sum() == tmp_paper_years.size)
#     if is_A == False:
#         continue
#     # 提取作者信息行
#     print("add:", row.author_id)
#     # tmp_author_info = authors.loc[authors['index'] == str(row.author_id)]
#     # print(tmp_author_info[['index','hi']])
#     # print(authors.loc[row.author_id-1, 'hi'])
#     # break
#     for epi_idx, epi_row in tmp_paper_ids.iterrows():
#         if epi_row.year >= 1984 and epi_row.year <= 1993:
#             print("add:", row.author_id, epi_row.paper_id)
#             A = A.append([{'author_id': epi_row.author_id,
#                            'hi': authors.loc[row.author_id-1, 'hi'], 'EPi10_paper_id':epi_row.paper_id}])
# # print(A)
# A.to_csv('A.csv')




# 得到新人数据的被引用量 EPi10
# 首先读取ap10
AP10 = pd.read_csv("ap10.csv", dtype={'refs': str})
ref_re = AP10[['index', 'refs']]
# ref_dict = {}
# 被引字典
refed_dict = {}
for index, row in ref_re.iterrows():
    if isinstance(row['refs'], float):
        continue
    _ref_table = row['refs'].split('|')

    # ref_dict[row['index']]=_ref_table
    for _ref in _ref_table:
        if _ref not in refed_dict:
            refed_dict[_ref] = []
        refed_dict[_ref].append(row['index'])


# import json
# with open('refed_re.json', 'w') as f:
#     json.dump(refed_dict, f)
# with open('ref_re.json', 'w') as f:
#     json.dump(ref_dict, f)


# # 遍历被引用关系，查看引用的每篇论文的年份，然后依次加到对应的中
# EPi_10 = {'paper_id':[], 'citationcount1':[], 'citationcount2':[]}
# for _paper_id in refed_dict:
#     tmp_citation_cnt1 = 0
#     tmp_citation_cnt2 = 0
#     for _refer_paper_id in refed_dict[_paper_id]:
#         _year = papers.loc[int(_refer_paper_id)-1, 'year']
#         if int(_year) <= 1988:
#             tmp_citation_cnt1+=1
#         else:
#             tmp_citation_cnt2+=1
#     EPi_10['paper_id'].append(_paper_id)
#     EPi_10['citationcount1'].append(tmp_citation_cnt1)
#     EPi_10['citationcount2'].append(tmp_citation_cnt2)
# EPi_10 = pd.DataFrame(EPi_10)
# EPi_10.to_csv("EPi_10.csv")


# 读取coauthor
coauthors = readCoauthor('AMiner-Coauthor.txt')
coauthors = pd.DataFrame(coauthors,dtype=np.int64)




Read Start: AMiner-Paper.txt
Read End: AMiner-Paper.txt
Read Start: AMiner-Author.txt
Read End: AMiner-Author.txt
Read Start: AMiner-Coauthor.txt
Read End: AMiner-Coauthor.txt


C:\Users\t-jintli\Anaconda3\lib\site-packages\numpy\core\numeric.py:2327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [3]:
# 读取B-paper分析
b_paper = pd.read_csv("b-paper.csv")
b_paper_ref_re = b_paper[['index']]
b_paper_ref_re = pd.merge(papers, b_paper_ref_re, on='index')[['index', 'refs', 'year']]

In [4]:
print(b_paper_ref_re.dtypes)

index     int64
refs     object
year     object
dtype: object


In [5]:
print(len(b_paper_ref_re['refs'].unique()))
# b_paper_ref_re = 

55229


In [6]:
# 直接用Author2Paper来进行筛选，会快很多
# 返回对应的b_paper中的论文id集合
def paper_in_b_paper(paper_ids):
    _papers = b_paper[b_paper['index'].isin(paper_ids)]
    return (_papers['index'].tolist())

# paper_cnt_citation_cnt_in_b_paper(['1', '2', '4'])

In [7]:
print(len(refed_dict))

63852


In [30]:
AP10 = pd.read_csv("ap10.csv", dtype={'refs': str})
ref_re = AP10[['index', 'refs']]
# ref_dict = {}
# 被引字典
refed_dict = {}
for index, row in ref_re.iterrows():
    if isinstance(row['refs'], float):
        continue
    _ref_table = row['refs'].split('|')

    # ref_dict[row['index']]=_ref_table
    for _ref in _ref_table:
        if _ref not in refed_dict:
            refed_dict[_ref] = []
        refed_dict[_ref].append(row['index'])

# b_paper_refed_dict表示某篇论文在b-paper被引的论文ID集合
# len(b_paper_refed_dict[paper_id])表示被引次数
b_paper_refed_dict = {}
for index, row in b_paper_ref_re.iterrows():
    if isinstance(row['refs'], float):
        continue
    _ref_table = row['refs'].split('|')
    for _ref in _ref_table:
        if _ref not in b_paper_refed_dict:
            b_paper_refed_dict[_ref] = []
        b_paper_refed_dict[_ref].append(row['index'])

In [22]:
print(len(b_paper_refed_dict))

76522


In [9]:

# DEBUG
i = 0
for k, v in refed_dict.items():
    print('-----------')
    print(type(k))
    print(k)
    print(len(v))
    i+=1
    if i ==3 :
        break

-----------
<class 'str'>
289259
3
-----------
<class 'str'>
289024
4
-----------
<class 'str'>
408638
546


In [28]:
print(A_paper.dtypes)

author_id         float64
EPi10_paper_id    float64
hi                  int64
year               object
refs               object
affiliations       object
dtype: object


In [10]:
EPi_10 = pd.read_csv("EPi_10.csv")
print(EPi_10.dtypes)

Unnamed: 0        int64
paper_id          int64
citationcount1    int64
citationcount2    int64
dtype: object


In [11]:
import csv

def read_csv_to_dict(path):
    with open(path, mode='r', encoding='utf8') as infile:
        reader = csv.reader(infile)
        mydict = {rows[1]:rows[0] for rows in reader}
    return mydict

ccf_rank = read_csv_to_dict("ccf.csv")
aff_rank = read_csv_to_dict("aff_rank.csv")

In [12]:
print(ccf_rank)

{'name': '\ufefflevel', 'Acta Informatica': 'C', 'Artificial Intelligence': 'A', 'Journal of the ACM': 'A', 'Computer-Aided Design': 'B', 'The Computer Journal': 'B', 'IEEE Transactions on Computers': 'A', 'IEEE Transactions on Information Theory': 'A', 'Information Processing Letters': 'C', 'Information Sciences': 'B', 'Journal of Computer and System Sciences': 'B', 'Networks': 'C', 'Pattern Recognition': 'B', 'Software: Practice and Experience': 'B', 'SIAM Journal on Computing': 'A', 'Computers & Graphics-UK': 'C', 'ACM Transactions on Mathematical Software': 'B', 'IEEE Transactions on Software Engineering': 'A', 'International Journal of Neural Systems': 'C', 'Computer Communications': 'C', 'Behaviour & Information Technology': 'C', 'IEEE Transactions on Pattern Analysis and Machine Intelligence': 'A', 'ACM Transactions on Programming Languages & Systems': 'A', 'Journal of Systems and Software': 'B', 'Fuzzy Sets and Systems': 'C', 'Performance Evaluation: An International Journal': 

In [14]:
SCAi = {'author':[], 'sc_author':[], 'papers':[], 'papers_cnt':[], 'citation_sum':[]}
FCAi = {'author':[], 'fc_author':[], 'papers':[], 'papers_cnt':[], 'citation_sum':[]}

MT = {'author_id':[], 'hi':[], 'paper_a5_cnt':[], 'paper_b5_cnt':[] , 'paper_cnt':[] , 'delta_paper_cnt':[] , 
      'ref_cnt':[], 'paper_a5_refed_cnt':[], 'paper_b5_refed_cnt':[], 'paper_refed_cnt':[], 'paper_refed_cnt_delta':[],
     'paper_avg_refed_cnt':[], 'c10':[], 'c30':[], 'pa':[], 'pb':[], 'pc':[], 'cn':[], 'scn':[], 'pscn':[], 'ccn':[], 'cscn':[],
     'cpcn':[], 'cpscn':[], 'aff_rank':[], 'H':[]}

# 学术新人集合
A = pd.read_csv("A.csv")
A_paper = pd.merge(papers, A, left_on='index', right_on='EPi10_paper_id')[['author_id', 'EPi10_paper_id', 'hi', 'year', 'refs', 'affiliations', 'publication_venue']]
EPi_10 = pd.read_csv("EPi_10.csv")


In [24]:
print(A_paper.dtypes)
print()
print(A.dtypes)


author_id            float64
EPi10_paper_id       float64
hi                     int64
year                  object
refs                  object
affiliations          object
publication_venue     object
dtype: object

Unnamed: 0          int64
author_id         float64
hi                  int64
EPi10_paper_id    float64
dtype: object


In [39]:
print(len(refed_dict))
print(refed_dict)

63852
{'289259': [10, 285857, 289459], '289024': [12, 8930, 88334, 150202], '408638': [12, 507, 555, 586, 836, 976, 1053, 1129, 1325, 1405, 1727, 1747, 1749, 2047, 2663, 2994, 3242, 3425, 3426, 3526, 3530, 3623, 3825, 3863, 3873, 3927, 3938, 4035, 4850, 5531, 6955, 8913, 8926, 8927, 8946, 9909, 9926, 11811, 11819, 12434, 12436, 12442, 12637, 13026, 13033, 15375, 15433, 15939, 17860, 17861, 17865, 19305, 19452, 19618, 19629, 19804, 20116, 20781, 22961, 23616, 23619, 23634, 23646, 23937, 24073, 24074, 24076, 24077, 24078, 24787, 24802, 24980, 26271, 26733, 27058, 27219, 27231, 27264, 27271, 27273, 27294, 27796, 31033, 31042, 31053, 31779, 32112, 32185, 32192, 32877, 32904, 34341, 35558, 35914, 36030, 36118, 36191, 36243, 36247, 36354, 37684, 38216, 38217, 38219, 38280, 38688, 38694, 38735, 39373, 41139, 41222, 41959, 43005, 43052, 44692, 45068, 45777, 46614, 46623, 47611, 51391, 51404, 54020, 54113, 54193, 54213, 54219, 54534, 54838, 54842, 54898, 55016, 55041, 55241, 55291, 55301, 55418

In [36]:
# print(A[['author_id','hi']].drop_duplicates())
DEBUG=True

i = 0
for _idx, _row in (A[['author_id', 'hi']].drop_duplicates().iterrows()):
    
    i += 1
    if i % 100 == 0:
        print("Processing progress:", i)
    
    # 新人id
    ai = _row['author_id']
    if DEBUG:
        print(ai)
    # 新人论文集合
    ai_paper = A_paper[A_paper['author_id'] == ai].drop_duplicates()
    #1
    paper_a5_cnt = ai_paper[ai_paper['year'] <= '1988'].shape[0]
    #2
    paper_b5_cnt = ai_paper[ai_paper['year'] > '1988'].shape[0]
#         print(ai_paper)

    #4
    delta_paper_cnt = paper_b5_cnt - paper_a5_cnt
    #3
    paper_cnt = paper_a5_cnt + paper_b5_cnt
    if DEBUG:
        print("a5 paper:",paper_a5_cnt)
        print("b5 paper:",paper_b5_cnt)
        print("total paper cnt:", paper_cnt)
        print("delta paper cnt:", delta_paper_cnt)
    
    #5
    ref_cnt = sum([len(str(ref).split("|")) if ref else 0 for ref in ai_paper['refs']])
    if DEBUG:
#         print([len(str(ref).split("|")) if ref else 0 for ref in ai_paper['refs']])
        print("total ref cnt:", ref_cnt)

    
    
    # 计算被引量
    #9
    refed_cnt = 0
    #6
    refed_a5_cnt = 0
    #7
    refed_b5_cnt = 0
    #11
    c10 = 0
    #12
    c30 = 0
    
    #14
    PA = 0
    #15
    PB = 0
    #16
    PC = 0
    #24 机构声望
    aff = ""
    # 根据年份和paper_id逐个paper统计被引量
    tmp_refed_table = []
    for __i, __paper in ai_paper.iterrows():
        __paper_id = __paper['EPi10_paper_id']
        __year = __paper['year']
        if DEBUG:
            print("paper id:", __paper_id)
        # 被引量的相关计算
        try:
            __tmp_paper_refed_cnt = len(refed_dict[str(__paper_id)])
            if DEBUG:
                print(__paper_id, " refed cnt:",__tmp_paper_refed_cnt)
            refed_cnt += __tmp_paper_refed_cnt
            if __year <= '1988':
                refed_a5_cnt += __tmp_paper_refed_cnt
            else:
                refed_b5_cnt += __tmp_paper_refed_cnt
            if __tmp_paper_refed_cnt >= 30:
                c30 += 1
                c10 += 1
            elif __tmp_paper_refed_cnt >= 10:
                c10 += 1
            tmp_refed_table.append(__tmp_paper_refed_cnt)
        except Exception as e:
            pass
        
        # 刊物rank
        _publication_venue = __paper['publication_venue']
        try:
            _venue_rank = ccf_rank(_publication_venue)
            if DEBUG:
                print(_publication_venue, " rank:", _venue_rank)
            if _venue_rank == 'A':
                PA+=1
            elif _venue_rank == 'B':
                PB+=1
            elif _venue_rank == 'C':
                PC+=1
        except Exception as e:
            pass
        
    # 计算h指数
    tmp_refed_table.sort(reverse=True)
    ai_H = 0
    for i, _ref in enumerate(tmp_refed_table):
        if i+1 >= _ref:
            ai_H = i
    
    #8 
    refed_delta = refed_b5_cnt - refed_a5_cnt
    
    #10 论文被引均数
    avg_refed_cnt = 0
    try:
        avg_refed_cnt = float(refed_cnt) / float(paper_cnt)
    except Exception as e:
        pass
    
    # 24 机构名声望分析
    aff = authors[authors['index'] == _row['author_id']]['affiliations'].unique()[0]
    aff = aff.split(';')
    aff_rank_ = 100000
    for _aff in aff:
        try:
            aff_rank = min(aff_rank_, aff_rank[_aff])
        except Exception as e:
            pass
        
    # 24 aff rank score
    if aff_rank_ == 100000:
        aff_rank_ = 0
    elif aff_rank_ >= 1 and aff_rank <= 200:
        aff_rank_ = 1
    elif aff_rank_ >= 201 and aff_rank <= 400:
        aff_rank_ = 2
    elif aff_rank_ >= 401 and aff_rank <= 600:
        aff_rank_ = 3
    elif aff_rank_ >= 601 and aff_rank <= 800:
        aff_rank_ = 4
    elif aff_rank_ >= 801:
        aff_rank_ = 5
    
    
    # 下面的被注释掉的没有跑
    # 17-23
    cn_ai = 0 # 合作者数量
    scn_ai = 0 # 强合作者数量
    scn_ai_paper_cnt = 0 # 强合作者发文数量
    
    ccn_ai_paper_refed_sum = 0 # 合作者被引总数
    scn_ai_paper_refed_sum = 0 # 强合作者被引总数
    
    # 强弱合作者分析
    # 找出合作作者
#     tmp_coauthors_p_1 = coauthors[coauthors['author1'] == ai]
#     tmp_coauthors_p_2 = coauthors[coauthors['author2'] == ai]
#     # 合作作者集合(似乎不用给出)
#     p = list(tmp_coauthors_p_1.author2.unique())
#     p += list(tmp_coauthors_p_2.author1.unique())
#     # 合作作者的论文集合
#     Tj1 = pd.merge(a2p, tmp_coauthors_p_1, left_on='author_id', right_on='author2')[['author2', 'paper_id']]
#     Tj1 = pd.merge(papers, Tj1, left_on='index', right_on='paper_id')[['author2', 'paper_id', 'year']]
#     Tj2 = pd.merge(a2p, tmp_coauthors_p_2, left_on='author_id', right_on='author1')[['author1', 'paper_id']]
#     Tj2 = pd.merge(papers, Tj2, left_on='index', right_on='paper_id')[['author1', 'paper_id', 'year']]
#     # 1993年之前发表的论文数量
#     for _co in p:
#         _Tj1 = Tj1[Tj1['author2'] == _co]
#         _Tj2 = Tj2[Tj2['author1'] == _co]
#         # 计算1993之前的论文数量
        _author_paper_b_1993 = (_Tj2.year <= '1993').sum() + \
                               (_Tj1.year <= '1993').sum()
        _paper_ids = paper_in_b_paper(_Tj1['paper_id'].tolist()) + paper_in_b_paper(_Tj2['paper_id'].tolist())
        _paper_ids = [str(_id) for _id in _paper_ids]
        if _author_paper_b_1993 >3:
            SCAi['author'].append(ai)
            SCAi['sc_author'].append(_co)
            SCAi['papers'].append(";".join(_paper_ids))
            SCAi['papers_cnt'].append(len(_paper_ids))
            try:
                _ci_sum = 0  
                for _id in _paper_ids:
                    try:
                        _ci_sum += len(b_paper_refed_dict[_id])
                    except:
                        pass
                SCAi['citation_sum'].append(_ci_sum)
                scn_ai_paper_refed_sum += _ci_sum
                ccn_ai_paper_refed_sum += _ci_sum
            except:
                SCAi['citation_sum'].append(0)

            scn_ai += 1
            cn_ai += 1
            scn_ai_paper_cnt += len(_paper_ids)
            
        
#         elif _author_paper_b_1993 >0:
#             FCAi['author'].append(ai)
#             FCAi['fc_author'].append(_co)
#             FCAi['papers'].append(";".join(_paper_ids))
#             FCAi['papers_cnt'].append(len(_paper_ids))
#             try:
#                 _ci_sum = sum([len(b_paper_refed_dict[_id]) for _id in _paper_ids])
#                 FCAi['citation_sum'].append(_ci_sum)
#                 ccn_ai_paper_refed_sum += _ci_sum
#             except:
#                 FCAi['citation_sum'].append(0)
            
#             cn_ai += 1
        
    # average refed cnt/person
    ccn_ai_paper_refed_avg_p = 0
    pccn_ai_paper_refed_avg_p = 0
    try:
        ccn_ai_paper_refed_avg_p = float(ccn_ai_paper_refed_sum)/float(cn_ai) # 合作者人均被引总数
        pccn_ai_paper_refed_avg_p = float(scn_ai_paper_refed_sum)/float(scn_ai_paper_cnt) # 强作者人均被引总数
    except Exception as e:
        pass
        

    MT['author_id'].append(ai)
    MT['hi'].append(_row['hi'])
    MT['paper_a5_cnt'].append(paper_a5_cnt)
    MT['paper_b5_cnt'].append(paper_b5_cnt)
    MT['delta_paper_cnt'].append(delta_paper_cnt)
    MT['paper_cnt'].append(paper_cnt)
    MT['ref_cnt'].append(ref_cnt)
    MT['paper_a5_refed_cnt'].append(refed_a5_cnt)
    MT['paper_b5_refed_cnt'].append(refed_b5_cnt)
    MT['paper_refed_cnt'].append(refed_cnt)
    MT['paper_refed_cnt_delta'].append(refed_delta)
    MT['paper_avg_refed_cnt'].append(avg_refed_cnt)
    MT['c10'].append(c10)
    MT['c30'].append(c30)
    MT['pa'].append(PA)
    MT['pb'].append(PB)
    MT['pc'].append(PC)
    MT['cn'].append(cn_ai)
    MT['scn'].append(scn_ai)
    MT['pscn'].append(scn_ai_paper_cnt)
    MT['ccn'].append(ccn_ai_paper_refed_sum)
    MT['cscn'].append(scn_ai_paper_cnt)
    MT['cpcn'].append(ccn_ai_paper_refed_avg_p)
    MT['cpscn'].append(pccn_ai_paper_refed_avg_p)
    MT['aff_rank'].append(aff_rank_)
    MT['H'].append(ai_H)
    break

print(MT)

112127.0
a5 paper: 10
b5 paper: 15
total paper cnt: 25
delta paper cnt: 5
total ref cnt: 97
paper id: 4.0
paper id: 2850.0
paper id: 25057.0
paper id: 43188.0
paper id: 47954.0
paper id: 55899.0
paper id: 57559.0
paper id: 58366.0
paper id: 60881.0
paper id: 66930.0
paper id: 79126.0
paper id: 82351.0
paper id: 93129.0
paper id: 95784.0
paper id: 112882.0
paper id: 116588.0
paper id: 154364.0
paper id: 268799.0
paper id: 291886.0
paper id: 415953.0
paper id: 454204.0
paper id: 464524.0
paper id: 556313.0
paper id: 692974.0
paper id: 1081098.0
{'author_id': [112127.0, 112127.0, 112127.0, 112127.0, 112127.0, 112127.0, 112127.0, 112127.0, 112127.0, 112127.0], 'hi': [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], 'paper_a5_cnt': [20, 20, 10, 10, 10, 10, 10, 10, 10, 10], 'paper_b5_cnt': [30, 30, 15, 15, 15, 15, 15, 15, 15, 15], 'paper_cnt': [50, 50, 25, 25, 25, 25, 25, 25, 25, 25], 'delta_paper_cnt': [10, 10, 5, 5, 5, 5, 5, 5, 5, 5], 'ref_cnt': [16, 16, 8, 8, 8, 8, 8, 8, 97, 9

In [13]:
print(aff_rank)

{'Organization': '\ufeffRank', 'IBM': '1', 'Microsoft': '2', 'Carnegie Mellon University': '3', 'Massachusetts Institute of Technology': '4', 'Stanford University': '5', 'AT&T': '6', 'University of Pittsburgh': '7', 'University of California, Berkeley': '8', 'Bell Laboratories': '9', 'Columbia University': '10', 'York University': '11', 'University of York': '12', 'University of Washington': '13', 'The University of Hong Kong': '14', 'University of Michigan': '15', 'New York University': '16', 'University of Illinois, Urbana-Champaign': '17', 'University of Pennsylvania': '18', 'INRIA': '19', 'Palo Alto Research Center': '20', 'Georgia Institute of Technology': '21', 'University of Maryland': '22', 'University of Toronto': '23', 'Cornell University': '24', 'University of California, Los Angeles': '25', 'CNRS': '26', 'Technion Israel Institute of Technology': '27', 'National University of Singapore': '28', 'Intel': '29', 'University of Texas, Austin': '30', 'University of London': '31',

In [57]:
print(MT)

{'author_id': [112127.0, 669143.0, 533344.0, 522333.0, 597188.0, 378500.0, 117256.0, 1223056.0, 94568.0], 'hi': [15.0, 3.0, 1.0, 1.0, 0.0, 0.0, 2.0, 2.0, 1.0], 'paper_a5_cnt': [20, 8, 1, 1, 1, 1, 4, 4, 1], 'paper_b5_cnt': [7, 7, 7, 7, 7, 7, 7, 7, 7], 'paper_cnt': [27, 15, 8, 8, 8, 8, 11, 11, 8], 'delta_paper_cnt': [-13, -1, 6, 6, 6, 6, 3, 3, 6], 'ref_cnt': [16, 1, 0, 0, 0, 1, 0, 1, 1], 'paper_a5_refed_cnt': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'paper_b5_refed_cnt': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'paper_refed_cnt': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'paper_refed_cnt_delta': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'paper_avg_refed_cnt': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'c10': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'c30': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'pa': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'pb': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'pc': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'cn': [22, 34, 0, 2, 2, 2, 3, 1, 1], 'scn': [12, 15, 0, 1, 1, 2, 3, 1, 1], 'pscn': [323, 249, 0, 13, 13, 13, 26, 4, 4], 'ccn': [0, 2, 0, 0, 0, 0, 0, 0, 0], 'csc

In [59]:
MT = pd.DataFrame(MT)
SCAi = pd.DataFrame(SCAi)
FCAi = pd.DataFrame(FCAi)
SCAi.to_csv("SCAi.csv")
FCAi.to_csv("FCAi.csv")
MT.to_csv("MT.csv")


In [ ]:

# 这里实现的是对每个学者查找它的强、弱合作者
# 然后储存在字典中，最后另存为csv
# _row['index']即表示学者id
i = 0
for _idx, _row in (authors.iterrows()):
    # 遍历各个学者
    i += 1
    if i % 100 == 0:
        print("Processing:", i)
    if i == 1000:
        break
    ai = _row['index']
    # 找出合作作者
    tmp_coauthors_p_1 = coauthors[coauthors['author1'] == _row['index']]
    tmp_coauthors_p_2 = coauthors[coauthors['author2'] == _row['index']]
    # 合作作者集合(似乎不用给出)
    p = list(tmp_coauthors_p_1.author2.unique())
    p += list(tmp_coauthors_p_2.author1.unique())
    # 合作作者的论文集合
    Tj1 = pd.merge(a2p, tmp_coauthors_p_1, left_on='author_id', right_on='author2')[['author2', 'paper_id']]
    Tj1 = pd.merge(papers, Tj1, left_on='index', right_on='paper_id')[['author2', 'paper_id', 'year']]
    Tj2 = pd.merge(a2p, tmp_coauthors_p_2, left_on='author_id', right_on='author1')[['author1', 'paper_id']]
    Tj2 = pd.merge(papers, Tj2, left_on='index', right_on='paper_id')[['author1', 'paper_id', 'year']]
    # 1993 年之后发表的论文数量
    # b_1993_sum = (Tj1.year <= 1984).sum() + (Tj2.year <= 1993).sum()
    # author_b_1993_paper_cnt = pd.pivot_table()

#     Tj1 = Tj1.groupby(['author2', 'year', 'paper_id'], as_index=False)['paper_id'].agg({'cnt':'count'})
#     Tj2 = Tj2.groupby(['author1', 'year', 'paper_id'], as_index=False)['paper_id'].agg({'cnt':'count'})

    for _co in p:
        _Tj1 = Tj1[Tj1['author2'] == _co]
        _Tj2 = Tj2[Tj2['author1'] == _co]
        # 计算1993之前的论文数量
        _author_paper_b_1993 = (_Tj2.year <= '1993').sum() + \
                               (_Tj1.year <= '1993').sum()
        _paper_ids = paper_in_b_paper(_Tj1['paper_id'].tolist()) + paper_in_b_paper(_Tj2['paper_id'].tolist())
        _paper_ids = [str(_id) for _id in _paper_ids]
        if _author_paper_b_1993 >3:
            SCAi['author'].append(_row['index'])
            SCAi['sc_author'].append(_co)
            SCAi['papers'].append(";".join(_paper_ids))
            SCAi['papers_cnt'].append(len(_paper_ids))
            try:
                SCAi['citation_sum'].append(sum([len(b_paper_refed_dict[_id]) for _id in _paper_ids]))
            except:
                SCAi['citation_sum'].append(0)
        elif _author_paper_b_1993 >0:
            FCAi['author'].append(_row['index'])
            FCAi['fc_author'].append(_co)
            FCAi['papers'].append(";".join(_paper_ids))
            FCAi['papers_cnt'].append(len(_paper_ids))
            try:
                FCAi['citation_sum'].append(sum([len(b_paper_refed_dict[_id]) for _id in _paper_ids]))
            except:
                FCAi['citation_sum'].append(0)

In [8]:
print(len(SCAi['author']))

1707
